# Summary of database import



## Set up

In [1]:
from pathlib import Path
import os
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
import pandas as pd

repodir = Path("../../") 
inputdir = repodir / "data/"
filename = repodir / 'secrets' / 'database.ini'
section = 'aws-lght-sl'


def read_dbparams(filename,section="postgresql"):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

def dbquery(query,dbparams):
    #print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**dbparams)
    cur = conn.cursor(cursor_factory=DictCursor)
    cur.execute(query)
    res = cur.fetchall()
    cur.close()
    if conn is not None:
        conn.close()
        #print('Database connection closed.')    
    return res



In [2]:
filename = repodir / 'secrets' / 'database.ini'
dbparams=read_dbparams(filename,section='aws-lght-sl')

## Define queries
Define queries we are going to use multiple times:

In [46]:
# Check comments on vocabularies
qry_vocabulary = "SELECT pg_catalog.obj_description(t.oid, 'pg_type')::json from pg_type t where typname = '%s';" 

# Number of records per source
qry_source = 'SELECT main_source,count(*) FROM litrev.%s GROUP BY main_source'
# Number of records per value of categorical variable
qry_values = ' select norm_value,count(*),count(distinct species),count(distinct species_code) from litrev.%s group by norm_value;'

# Number of records per value of numerical variable
qry_triplet = ' select best is NULL as b, lower is NULL as l, upper is NULL as u,count(*),count(distinct species),count(distinct species_code) from litrev.%s group by b,l,u;'

# Raw values when norm value is NULL 
qry_nulls = ' select raw_value,count(*),count(distinct species),count(distinct species_code) from litrev.%s where norm_value is NULL group by raw_value;'
# Raw values when best/lower/upper are all NULL 
qry_triplet_nulls = 'select raw_value,count(*),count(distinct species),count(distinct species_code) from litrev.%s where best is NULL and lower is NULL and upper is NULL group by raw_value;'


This is a complex query for the comments in one column:

In [4]:
qry_comment = """
SELECT
    (
        SELECT
            pg_catalog.col_description(c.oid, cols.ordinal_position::int)
        FROM pg_catalog.pg_class c
        WHERE
            c.oid     = (SELECT CONCAT(cols.table_schema,'.',cols.table_name)::regclass::oid) AND
            c.relname = cols.table_name
    )::json as column_comment
 
FROM information_schema.columns cols
WHERE
    cols.table_catalog = 'dbfireveg' AND
    cols.table_schema  = 'litrev' AND
    cols.table_name    = '%s' AND
    cols.column_name    = 'best';   
""" 

## Read table with trait information

In [5]:
qry = "select code,name,description,value_type,life_stage,life_history_process,category_vocabulary,method_vocabulary from litrev.trait_info ;"
res = dbquery(qry,dbparams)
data = pd.DataFrame(res)
trait_info=data.rename(columns={0:"Trait code", 1:"Trait name", 2:"Description", 4:"Life stage", 5:"Life history process",  3:"value_type",6:"category_vocabulary",7:"method_vocabulary"})


## First tranche

### Trait code surv1

Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others| Data curation
---|---|---|---|---|---|---
<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>|<font color='darkgreen'>11563 records</font>|<font color='darkgreen'>18344 records</font>|None|<font color='red'>43 records to review</font> 

#### Trait information

Check trait information for this trait:

In [6]:
elem = trait_info[trait_info["Trait code"]=="surv1"]
elem.transpose()

,31
Trait code,surv1
Trait name,Resprouting - full canopy scorch
Description,Ordinal categories of survival and resprouting...
value_type,categorical
Life stage,Standing plant
Life history process,Survival
category_vocabulary,resprouting_vocabulary
method_vocabulary,method_surv1_vocabulary


#### Vocabularies

This trait uses one vocabulary for the categories and one for the methods

In [7]:
#For the category
res = dbquery(qry_vocabulary %  elem.iloc[0]['category_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
None,< 5 % individuals in a population resprout aft...
Few,> 5 and < 30 % individuals in a population res...
Half,> 30 and < 70 % individuals in a population re...
Most,> 70 and < 90 % individuals in a population re...
All,> 90 % individuals in a population resprout af...
Unknown,No data


In [8]:
#For the methods
res = dbquery(qry_vocabulary %  elem.iloc[0]['method_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
Direct field observation or measure (unknown sample size),Estimates based on data observed or measured i...
Direct observation (small sample),Estimates based on data observed or measured i...
Direct observation (large sample),Estimates based on data observed or measured i...
Estimated by extrapolation from observed values,Estimated by extrapolation from (time series) ...
Inferred from plant / organ / growth stage morphology,Inference based on relevant morphological stru...
Inferred from related taxa,Assigned to same category as congeners or conf...
Manipulative experiment,Manipulative experiment. Individuals sampled b...
Qualitative estimate,Guestimate from visual assessment of post-fire...
Unspecified methods,Methods unclear or undocumented.


#### Importing records

I ran the script in notebook '[Read resprouting data from NSWFFRD](python/Literature-review/Read-resprouting-data-from-NSWFFRD.ipynb)'. It imported 11563 records. Then I ran the script in notebook [Read resprouting from Austraits data](python/Literature-review/Read-resprouting-from-Austraits-data.ipynb) and it imported more than 18344 records.

#### Check records in database

In [56]:
res=dbquery(qry_source % 'surv1',dbparams)
pd.DataFrame(res)

,0,1
0,NSWFFRDv2.1,11563
1,austraits-3.0.2,18344


In [57]:
res = dbquery(qry_values % 'surv1',dbparams)
data=pd.DataFrame(res)
data.rename(columns={0:"Value",1:"Nr. records",2:"Nr. taxa",3:"Nr. valid"}) 

,Value,Nr. records,Nr. taxa,Nr. valid
0,None,12389,5600,4050
1,Few,254,181,181
2,Half,773,542,533
3,Most,244,163,163
4,All,16189,6410,3963
5,Unknown,15,14,14
6,None,43,36,31


Few missing values, they all fall in similar categories:

In [58]:
res=dbquery(qry_nulls % 'surv1',dbparams)
pd.DataFrame(res)

,0,1,2,3
0,"[fire_response, fire_killed fire_not_relevant,...",11,7,6
1,"[fire_response, fire_not_relevant not_fire_kil...",4,4,4
2,"[fire_response, fire_not_relevant resprouts, e...",28,25,21


### Trait code surv4

Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others | Data curation
---|---|---|---|---|---|---
<font color='red'>Description missing</font>|<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>| <font color='darkgreen'>1411 records</font> | TO DO | None | <font color='red'>TO DO: ca. 20 records </font>

#### Trait information

Check trait information

In [10]:
elem = trait_info[trait_info["Trait code"]=="surv4"]
elem.transpose()

,11
Trait code,surv4
Trait name,Regenerative Organ
Description,None
value_type,categorical
Life stage,Standing plant
Life history process,Survival
category_vocabulary,resprout_organ_vocabulary
method_vocabulary,None


#### Vocabularies

This trait uses one vocabulary for the categories but does not include a methods vocabulary:

In [11]:
#For the category
res = dbquery(qry_vocabulary %  elem.iloc[0]['category_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
Epicormic,Resprouting from epicormic meristematic tissue...
Apical,Resprouting from active shoots protected by cr...
Lignotuber,Resprouting from meristematic tissues or buds ...
Basal,Resprouting from meristematic tissues or buds ...
Tuber,Resprouting from non-woody nodular subsoil org...
Tussock,Shoots protected and resprouting within tightl...
Long rhizome or root sucker,Resprouting from buried woody or non-woody hor...
Short rhizome,Resprouting from buried woody or non-woody hor...
Stolon,Resprouting from non-woody horizontal organs o...
None,No vegetative recovery - killed by fire


#### Importing records

I ran the script in notebook '[Read categorical variables from NSWFFRD](python/Literature-review/Read-categorical-variables-from-NSWFFRD.ipynb)' for several variables. For _surv4_ it imported 1411 records, only 18 records had errors and could not be assigned to one of the predefined categories.


#### Check records in database

In [12]:
res = dbquery(qry_values % 'surv4',dbparams)
data=pd.DataFrame(res)
data.rename(columns={0:"Value",1:"Nr. records",2:"Nr. taxa",3:"Nr. valid"}) 

,Value,Nr. records,Nr. taxa,Nr. valid
0,Epicormic,147,146,146
1,Apical,14,14,14
2,Lignotuber,157,157,157
3,Basal,630,630,613
4,Tuber,104,104,104
5,Short rhizome,166,165,165
6,Long rhizome or root sucker,172,171,171
7,Stolon,3,3,3
8,None,18,17,17


Summary of raw values when norm value is NULL:

In [13]:
res=dbquery(qry_nulls % 'surv4',dbparams)
pd.DataFrame(res)

,0,1,2,3
0,"[Resprout location, apical buds]",4,4,4
1,"[Resprout location, basal, possibly rhizome, -...",1,1,1
2,"[Resprout location, basal rhizome]",1,1,1
3,"[Resprout location, basal & some epicormic, ->...",1,1,1
4,"[Resprout location, basal; stem buds; swamp fo...",1,1,1
5,"[Resprout location, basal: tuber (10; 48), ->,...",1,1,1
6,"[Resprout location, has rhizome]",2,2,2
7,"[Resprout location, or rhizome]",1,1,1
8,"[Resprout location, rhizome?]",1,1,1
9,"[Resprout location, roots]",2,2,2


### repr3
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others | Data curation
---|---|---|---|---|---|---
<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>| <font color='darkgreen'>838 records</font>| <font color='red'>TO DO</font> | None | <font color='red'>49 records</font>

#### Trait information

In [14]:
elem = trait_info[trait_info["Trait code"]=="repr3"]
elem.transpose()

,36
Trait code,repr3
Trait name,Age at first flower production (from seed)
Description,The time taken for first individual in a recru...
value_type,numerical
Life stage,Standing plant
Life history process,Reproduction
category_vocabulary,None
method_vocabulary,method_repr3_vocabulary


#### Vocabularies

In [15]:
res=dbquery(qry_comment % 'repr3',dbparams)
pd.DataFrame(res[0]).transpose()

,0
years,The time taken for first individual in a recru...


In [16]:
#For the method
res = dbquery(qry_vocabulary %  elem.iloc[0]['method_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
Direct field observation or measure (unknown sample size),Estimates based on data observed or measured i...
Direct observation (small sample),Estimates based on data observed or measured i...
Direct observation (large sample),Estimates based on data observed or measured i...
"Trait evidence, but start date uncertain","Evidence of trait expression, but age of first..."
No evidence of trait expression by specified age,No evidence of trait expression by specified a...
Unspecified methods,Methods unclear or undocumented.


#### Importing records

I ran the script in notebook '[Read numerical variables from NSWFFRD](python/Literature-review/Read-numerical-variables-from-NSWFFRD.ipynb)' for several variables. For _surv4_ it imported 838 records, but 81 records had errors and could not be assigned to a fuzzy number.


#### Check records in database
This query will show how many records have null values in each colum (a line with 'True True True' means raw values could not be translated to a triplet:

In [17]:
res = dbquery(qry_triplet % 'repr3',dbparams)
data=pd.DataFrame(res)
data.rename(columns={0:"Best",1:"Lower",2:"Upper",3:"Nr. records",4:"Nr. taxa",5:"Nr. valid"}) 

,Best,Lower,Upper,Nr. records,Nr. taxa,Nr. valid
0,False,True,True,546,532,528
1,True,False,False,142,138,138
2,True,False,True,29,29,29
3,True,True,False,72,72,72
4,True,True,True,49,47,47


In [18]:
res=dbquery(qry_triplet_nulls % 'repr3',dbparams)
pd.DataFrame(res)

,0,1,2,3
0,"[Primary juvenile period, ]",1,1,1
1,"[Primary juvenile period, 0.25-0.5]",1,1,1
2,"[Primary juvenile period, 0.3]",1,1,1
3,"[Primary juvenile period, <10% flowering at 4 ...",1,1,1
4,"[Primary juvenile period, 15% produce seed at 1y]",1,1,1
5,"[Primary juvenile period, 2;]",1,1,1
6,"[Primary juvenile period, 22 months]",1,1,1
7,"[Primary juvenile period, 2; 3 (138)]",1,1,1
8,"[Primary juvenile period, 2.5 (1b)]",1,1,1
9,"[Primary juvenile period, 2 for first flowerin...",1,1,1


### repr4
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others
---|---|---|---|---|---
<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>| <font color='red'>TO DO</font>| <font color='red'>TO DO</font>  | None 

#### Trait information

In [19]:
elem = trait_info[trait_info["Trait code"]=="repr4"]
elem.transpose()

,1
Trait code,repr4
Trait name,Maturation age
Description,The time taken for 50% of individuals in a coh...
value_type,numerical
Life stage,Standing plant
Life history process,Reproduction
category_vocabulary,None
method_vocabulary,method_repr4_vocabulary


#### Vocabularies

In [20]:
res=dbquery(qry_comment % 'repr4',dbparams)
pd.DataFrame(res[0]).transpose()

,0
years,The time taken for 50% of individuals in a coh...


In [21]:
#For the method
res = dbquery(qry_vocabulary %  elem.iloc[0]['method_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
Direct field observation or measure (unknown sample size),Estimates based on data observed or measured i...
Direct observation (small sample),Estimates based on data observed or measured i...
Direct observation (large sample),Estimates based on data observed or measured i...
Estimated by extrapolation from observed values,Estimated by extrapolation from (time series) ...
"Trait evidence, but start date uncertain","Evidence of trait expression, but age of first..."
Inferred form other trait,"Inferred from values of a different, but relat..."
Inferred from partial data,Inferred from incomplete or indirect informati...
No evidence of trait expression by specified age,No evidence of trait expression by specified a...
Unspecified methods,Methods unclear or undocumented.


In [22]:
res = dbquery(qry_triplet % 'repr4',dbparams)
data=pd.DataFrame(res)
data.rename(columns={0:"Best",1:"Lower",2:"Upper",3:"Nr. records",4:"Nr. taxa",5:"Nr. valid"}) 

""


### germ8
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others
---|---|---|---|---|---
<font color='red'>TO DO: update description</font>|<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>| No records| <font color='red'>TO DO</font>  | None 

#### Trait information

In [23]:
elem = trait_info[trait_info["Trait code"]=="germ8"]
elem.transpose()

,3
Trait code,germ8
Trait name,Seed dormancy type
Description,None
value_type,categorical
Life stage,Seed
Life history process,Germination
category_vocabulary,seed_dormancy_vocabulary
method_vocabulary,method_germ8_vocabulary


#### Vocabularies

In [24]:
#For the category
res = dbquery(qry_vocabulary %  elem.iloc[0]['category_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
PY,Physical dormancy (PY) is caused by a water-im...
PD,Physiological dormancy (PD) is caused by a che...
PY-PD,A combination of physical dormancy (PY) and ph...
MPD,"A combination of morphological dormancy (MD), ..."
MD,Morphological dormancy (MD) is caused by an un...
ND,Non-dormant (ND) species are able to germinate...


In [25]:
#For the methods
res = dbquery(qry_vocabulary %  elem.iloc[0]['method_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
Direct field observation or measure (unknown sample size),Estimates based on data observed or measured i...
Direct observation (small sample),Estimates based on data observed or measured i...
Direct observation (large sample),Estimates based on data observed or measured i...
Inferred from plant / organ / growth stage morphology,Inference based on relevant morphological stru...
Inferred from related taxa,Assigned to same category as congeners or conf...
Unspecified methods,Methods unclear or undocumented.


## Second tranche

### repr2
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others | Data curation
---|---|---|---|---|---|---
<font color='red'>TO DO: update description</font>|<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>| <font color='darkgreen'>144 records</font>|  TO DO | None | <font color='red'>TO DO: ca. 20 records</font>


#### Trait information

In [26]:
elem = trait_info[trait_info["Trait code"]=="repr2"]
elem.transpose()

,2
Trait code,repr2
Trait name,Post-fire flowering response
Description,None
value_type,categorical
Life stage,Standing plant
Life history process,Reproduction
category_vocabulary,postfire_response
method_vocabulary,None


#### Vocabularies

This trait uses one vocabulary for the categories but does not include a methods vocabulary:

In [27]:
#For the category
res = dbquery(qry_vocabulary %  elem.iloc[0]['category_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
Exclusive,Flowering occurs exclusively in the first 5 ye...
Facultative,Flowering occurs most prolifically in the firs...
Negligible,Flowering similar or variable throughout the f...
Unknown,No data


#### Importing records

I ran the script in notebook '[Read categorical variables from NSWFFRD](/lab/tree/python/Literature-review/Read-categorical-variables-from-NSWFFRD.ipynb)' for several variables. For _repr2_ it imported 144 records, ca. 120 species in two categories, and ca. 23 records had errors and could not be assigned to one of the predefined categories. No records for class 'Negligible' or 'Unknown'.


#### Check records in database

In [28]:
res = dbquery(qry_values % 'repr2',dbparams)
data=pd.DataFrame(res)
data.rename(columns={0:"Value",1:"Nr. records",2:"Nr. taxa",3:"Nr. valid"}) 

,Value,Nr. records,Nr. taxa,Nr. valid
0,Exclusive,30,30,30
1,Facultative,91,91,91
2,None,23,21,21


### grow1
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others
---|---|---|---|---|---
<font color='darkgreen'>Done</font>|TO DO|TO DO| TO DO| TO DO | None 


#### Trait information

In [29]:
elem = trait_info[trait_info["Trait code"]=="grow1"]
elem.transpose()

,34
Trait code,grow1
Trait name,Age to develop regenerative/resistance organs
Description,The time taken to develop organs or tissues en...
value_type,numerical
Life stage,Standing plant
Life history process,Growth
category_vocabulary,None
method_vocabulary,method_grow1_vocabulary


#### Vocabularies

In [30]:
#For the category
res = dbquery(qry_vocabulary %  elem.iloc[0]['method_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
Direct field observation or measure (unknown sample size),Estimates based on data observed or measured i...
Direct observation (small sample),Estimates based on data observed or measured i...
Direct observation (large sample),Estimates based on data observed or measured i...
Estimated by extrapolation from observed values,Estimated by extrapolation from (time series) ...
"Trait evidence, but start date uncertain","Evidence of trait expression, but age of first..."
Inferred from plant / organ / growth stage morphology,Inference based on relevant morphological stru...
Inferred from related taxa,Assigned to same category as congeners or conf...
Manipulative experiment,Manipulative experiment. Experimental heating ...
No evidence of trait expression by specified age,No evidence of trait expression by specified a...
Unspecified methods,Methods unclear or undocumented.


### rect2
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others | Data curation
---|---|---|---|---|---|---
<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>| <font color='darkgreen'>Done</font>| TO DO | None | <font color='red'>TO DO: >90 records</font>


#### Trait information

In [31]:
elem = trait_info[trait_info["Trait code"]=="rect2"]
elem.transpose()

,14
Trait code,rect2
Trait name,Establishment pattern
Description,The temporal pattern of seedling or clonal est...
value_type,categorical
Life stage,Seedling
Life history process,Recruitment
category_vocabulary,establishment_vocabulary
method_vocabulary,None


#### Vocabularies

In [32]:
res = dbquery(qry_vocabulary %  elem.iloc[0]['category_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
Intolerant,I- intolerant of conditions in established com...
Intolerant-Tolerant,I/T- intolerant or tolerant of conditions in e...
Tolerant,T- tolerant of conditions through the fire cyc...
Tolerant-Requiring,T/R- tolerant or requiring of conditions in es...
Requiring,R- requiring of of conditions in established c...
Unknown,No data


#### Importing records

I ran the script in notebook '[Read categorical variables from NSWFFRD](python/Literature-review/Read-categorical-variables-from-NSWFFRD.ipynb)' for several variables. For _rect2_ it imported 1088 records, >900 species in four categories, and ca. 96 records had errors and could not be assigned to one of the predefined categories. 


#### Check records in database

In [33]:
dbquery(qry_values % 'rect2',dbparams)

[['Intolerant', 636, 634, 634],
 ['Intolerant-Tolerant', 3, 3, 3],
 ['Tolerant', 318, 318, 318],
 ['Requiring', 35, 34, 34],
 [None, 96, 92, 92]]

### germ1
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others | Data curation
---|---|---|---|---|---|--- 
<font color='red'>TO DO: add description</font>|<font color='darkgreen'>Done</font>|<font color='darkgreen'>Done</font>| <font color='darkgreen'>Done: 1635 imported</font>| TO DO | None | <font color='red'>TO DO: >80 records</font>



#### Trait information

In [34]:
elem = trait_info[trait_info["Trait code"]=="germ1"]
elem.transpose()

,5
Trait code,germ1
Trait name,Seedbank Type
Description,None
value_type,categorical
Life stage,Seed
Life history process,Germination
category_vocabulary,seedbank_vocabulary
method_vocabulary,None


In [35]:
dbquery(qry_vocabulary % elem['category_vocabulary'],dbparams)

[]

#### Importing records

I ran the script in notebook '[Read categorical variables from NSWFFRD](python/Literature-review/Read-categorical-variables-from-NSWFFRD.ipynb)' for several variables. For _rect2_ it imported 1635 records, >1300 species in four categories, and >80 records had errors and could not be assigned to one of the predefined categories. 


#### Check records in database

In [36]:
dbquery(qry_values % 'germ1',dbparams)

[['Soil-persistent', 962, 958, 957],
 ['Transient', 267, 267, 266],
 ['Canopy', 228, 226, 220],
 ['Non-canopy', 95, 95, 90],
 [None, 83, 82, 82]]

### repr3a
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others | Data curation
---|---|---|---|---|---|---
<font color='darkgreen'>Done</font>|TO DO|TO DO| TO DO| TO DO | None | <font color='red'>TO DO: >80 records</font>



#### Trait information

In [37]:
elem = trait_info[trait_info["Trait code"]=="repr3a"]
elem.transpose()

,37
Trait code,repr3a
Trait name,Time to first postfire reproduction (from resp...
Description,The time taken after fire for first reproducti...
value_type,numerical
Life stage,Standing plant
Life history process,Reproduction
category_vocabulary,None
method_vocabulary,method_repr3a_vocabulary


In [38]:
res = dbquery(qry_vocabulary %  elem.iloc[0]['method_vocabulary'],dbparams)
pd.DataFrame(res[0]).transpose()

,0
Direct field observation or measure (unknown sample size),Estimates based on data observed or measured i...
Direct observation (small sample),Estimates based on data observed or measured i...
Direct observation (large sample),Estimates based on data observed or measured i...
"Trait evidence, but start date uncertain","Evidence of trait expression, but age of first..."
No evidence of trait expression by specified age,No evidence of trait expression by specified a...
Unspecified methods,Methods unclear or undocumented.


In [39]:
res = dbquery(qry_triplet % 'repr3a',dbparams)
data=pd.DataFrame(res)
data.rename(columns={0:"Best",1:"Lower",2:"Upper",3:"Nr. records",4:"Nr. taxa",5:"Nr. valid"}) 

,Best,Lower,Upper,Nr. records,Nr. taxa,Nr. valid
0,False,True,True,394,392,383
1,True,False,False,38,38,38
2,True,False,True,19,19,19
3,True,True,False,124,124,124
4,True,True,True,87,87,87


## Third tranche

### surv5
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others | Data curation
---|---|---|---|---|---|---
<font color='darkgreen'>Done</font>|TO DO|TO DO|1262 records| TO DO| TO DO | <font color='red'>>500 (!)</font>


#### Trait information

In [40]:
elem = trait_info[trait_info["Trait code"]=="surv5"]
elem.transpose()

,12
Trait code,surv5
Trait name,Standing plant longevity (Max)
Description,Age at which 50% of individuals in a cohort (e...
value_type,numerical
Life stage,Standing plant
Life history process,Survival
category_vocabulary,None
method_vocabulary,None


### surv6
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others | Data curation
---|---|---|---|---|---|---
<font color='darkgreen'>Done</font>|TO DO|TO DO|TO DO| TO DO| TO DO | None 


#### Trait information

In [41]:
elem = trait_info[trait_info["Trait code"]=="surv6"]
elem.transpose()

,13
Trait code,surv6
Trait name,Seedbank half-life
Description,Age at which 50% of a seed cohort in an in sit...
value_type,numerical
Life stage,Seed
Life history process,Survival
category_vocabulary,None
method_vocabulary,None


### surv7
Metadata | Structure | Vocabularies | Records NSWFFRD | Records austraits | Others | Data curation
---|---|---|---|---|---|---
<font color='darkgreen'>Done</font>|TO DO|TO DO|87 records| TO DO| TO DO | half of them (!) 


#### Trait information

In [42]:
elem = trait_info[trait_info["Trait code"]=="surv7"]
elem.transpose()

,0
Trait code,surv7
Trait name,Seed longevity
Description,Age at which all seeds in a cohort (excluding ...
value_type,numerical
Life stage,Seed
Life history process,Survival
category_vocabulary,None
method_vocabulary,None
